# 100k DOFs case

From the Kratos outputs it seems that on every "Fluid Dynamics Analysis" step 6
problems are solved using AMGCL: 2 larger ones, and 4 smaller ones (a larger
problem has exactly 4 times more DOFs than a smaller one). Riccardo says these
are all physically different problems, which seems to be confirmed by the fact
that each of the problems takes distinctive and repeated across time steps
number of iterations to converge. I'll consider each of these as a separate
sequence of problems.

AMGCL parameters:

- `solver.type=bicgstab`
- `solver.tol=1e-6`
- `precond.coarse_enough=500`
- `precond.coarsening.type=aggregation`
- `precond.relax.type=damped_jacobi`

The larger problems were solved with 4x4 block values both with OpenMP and VexCL(CUDA) backends.

In [1]:
import pandas
import re
from pathlib import Path
def csv(fname):
    return pandas.read_csv(fname, delimiter='\t', header=None)

def speedup(full, part):
    data = {}
    for fname in (full, part):
        for line in open(fname):
            if m := re.search(r'amgcl:\s+(\d+\.\d+) s', line.strip()):
                data[(fname, 'total')] = float(m.group(1))
            elif m := re.search(r'setup:\s+(\d+\.\d+) s', line.strip()):
                data[(fname, 'setup')] = float(m.group(1))
    print(f"Total speedup: {100 * (data.get((full, 'total'), 0) / data.get((part, 'total'), 1) - 1):.2f}%")
    print(f"Setup speedup: {100 * (data.get((full, 'setup'), 0) / data.get((part, 'setup'), 1) - 1):.2f}%")
    
def report(size, device, num, mode):
    full = Path(f'{size}/{device}-{num}-none.time')
    part = Path(f'{size}/{device}-{num}-{mode}.time')
    
    print('*** Full AMGCL rebuild on every step ***\n')
    print(full.read_text().strip())
    print(f"\nAverage iterations: {csv(full.with_suffix('.log')).mean()[2]:.1f}")
    
    print('\n\n*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***\n')
    print(part.read_text().strip())
    print(f"\nAverage iterations: {csv(part.with_suffix('.log')).mean()[2]:.1f}")
    speedup(full, part)

## Small problem 0, OpenMP

In [2]:
report('small', 'cpu', 0, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:        4.254 s] (100.00%)
[ self:          0.015 s] (  0.35%)
[  amgcl:        3.925 s] ( 92.27%)
[    setup:      1.145 s] ( 26.92%)
[      full:     1.145 s] ( 26.92%)
[    solve:      2.780 s] ( 65.35%)
[  reading:      0.314 s] (  7.38%)

Average iterations: 5.0


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:       3.548 s] (100.00%)
[ self:         0.062 s] (  1.74%)
[  amgcl:       3.224 s] ( 90.89%)
[    setup:     0.026 s] (  0.73%)
[    solve:     3.198 s] ( 90.16%)
[  reading:     0.261 s] (  7.37%)

Average iterations: 5.5
Total speedup: 21.74%
Setup speedup: 4303.85%


In [3]:
report('small', 'cpu', 0, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:        4.254 s] (100.00%)
[ self:          0.015 s] (  0.35%)
[  amgcl:        3.925 s] ( 92.27%)
[    setup:      1.145 s] ( 26.92%)
[      full:     1.145 s] ( 26.92%)
[    solve:      2.780 s] ( 65.35%)
[  reading:      0.314 s] (  7.38%)

Average iterations: 5.0


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           3.793 s] (100.00%)
[ self:             0.061 s] (  1.62%)
[  amgcl:           3.472 s] ( 91.54%)
[    setup:         0.703 s] ( 18.53%)
[      full:        0.118 s] (  3.11%)
[      partial:     0.585 s] ( 15.43%)
[    solve:         2.769 s] ( 73.00%)
[  reading:         0.260 s] (  6.84%)

Average iterations: 5.0
Total speedup: 13.05%
Setup speedup: 62.87%


## Small problem 0, VexCL(CUDA)

In [4]:
report('small', 'gpu', 0, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:        2.940 s] (100.00%)
[ self:          0.008 s] (  0.28%)
[  amgcl:        2.671 s] ( 90.85%)
[    setup:      1.890 s] ( 64.29%)
[      full:     1.890 s] ( 64.29%)
[    solve:      0.781 s] ( 26.56%)
[  reading:      0.261 s] (  8.87%)

Average iterations: 5.0


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:       1.682 s] (100.00%)
[ self:         0.489 s] ( 29.08%)
[  amgcl:       0.939 s] ( 55.82%)
[    setup:     0.038 s] (  2.28%)
[    solve:     0.901 s] ( 53.54%)
[  reading:     0.254 s] ( 15.10%)

Average iterations: 5.5
Total speedup: 184.45%
Setup speedup: 4873.68%


In [5]:
report('small', 'gpu', 0, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:        2.940 s] (100.00%)
[ self:          0.008 s] (  0.28%)
[  amgcl:        2.671 s] ( 90.85%)
[    setup:      1.890 s] ( 64.29%)
[      full:     1.890 s] ( 64.29%)
[    solve:      0.781 s] ( 26.56%)
[  reading:      0.261 s] (  8.87%)

Average iterations: 5.0


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           2.334 s] (100.00%)
[ self:             0.008 s] (  0.33%)
[  amgcl:           2.065 s] ( 88.48%)
[    setup:         1.284 s] ( 55.02%)
[      full:        0.189 s] (  8.09%)
[      partial:     1.096 s] ( 46.93%)
[    solve:         0.781 s] ( 33.46%)
[  reading:         0.261 s] ( 11.18%)

Average iterations: 5.0
Total speedup: 29.35%
Setup speedup: 47.20%


## Small problem 2, OpenMP

In [6]:
report('small', 'cpu', 2, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:       11.078 s] (100.00%)
[ self:          0.066 s] (  0.60%)
[  amgcl:       10.753 s] ( 97.07%)
[    setup:      1.246 s] ( 11.24%)
[      full:     1.246 s] ( 11.24%)
[    solve:      9.508 s] ( 85.82%)
[  reading:      0.258 s] (  2.33%)

Average iterations: 15.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:      11.213 s] (100.00%)
[ self:         0.066 s] (  0.59%)
[  amgcl:      10.879 s] ( 97.03%)
[    setup:     1.257 s] ( 11.21%)
[    solve:     9.622 s] ( 85.81%)
[  reading:     0.268 s] (  2.39%)

Average iterations: 15.7
Total speedup: -1.16%
Setup speedup: -0.88%


In [7]:
report('small', 'cpu', 2, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:       11.078 s] (100.00%)
[ self:          0.066 s] (  0.60%)
[  amgcl:       10.753 s] ( 97.07%)
[    setup:      1.246 s] ( 11.24%)
[      full:     1.246 s] ( 11.24%)
[    solve:      9.508 s] ( 85.82%)
[  reading:      0.258 s] (  2.33%)

Average iterations: 15.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:          10.380 s] (100.00%)
[ self:             0.062 s] (  0.59%)
[  amgcl:          10.057 s] ( 96.89%)
[    setup:         0.749 s] (  7.22%)
[      full:        0.128 s] (  1.23%)
[      partial:     0.622 s] (  5.99%)
[    solve:         9.308 s] ( 89.67%)
[  reading:         0.261 s] (  2.51%)

Average iterations: 15.6
Total speedup: 6.92%
Setup speedup: 66.36%


## Small problem 2, VexCL(CUDA)

In [8]:
report('small', 'gpu', 2, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:        4.811 s] (100.00%)
[ self:          0.008 s] (  0.16%)
[  amgcl:        4.540 s] ( 94.35%)
[    setup:      1.904 s] ( 39.57%)
[      full:     1.904 s] ( 39.57%)
[    solve:      2.636 s] ( 54.78%)
[  reading:      0.264 s] (  5.48%)

Average iterations: 15.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:       5.283 s] (100.00%)
[ self:         0.469 s] (  8.88%)
[  amgcl:       4.556 s] ( 86.24%)
[    setup:     1.914 s] ( 36.24%)
[    solve:     2.642 s] ( 50.00%)
[  reading:     0.258 s] (  4.88%)

Average iterations: 15.7
Total speedup: -0.35%
Setup speedup: -0.52%


In [9]:
report('small', 'gpu', 2, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:        4.811 s] (100.00%)
[ self:          0.008 s] (  0.16%)
[  amgcl:        4.540 s] ( 94.35%)
[    setup:      1.904 s] ( 39.57%)
[      full:     1.904 s] ( 39.57%)
[    solve:      2.636 s] ( 54.78%)
[  reading:      0.264 s] (  5.48%)

Average iterations: 15.7


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:           4.225 s] (100.00%)
[ self:             0.008 s] (  0.18%)
[  amgcl:           3.953 s] ( 93.55%)
[    setup:         1.340 s] ( 31.70%)
[      full:        0.205 s] (  4.85%)
[      partial:     1.135 s] ( 26.85%)
[    solve:         2.613 s] ( 61.84%)
[  reading:         0.265 s] (  6.27%)

Average iterations: 15.6
Total speedup: 14.85%
Setup speedup: 42.09%


## Large problem 0, OpenMP

In [10]:
report('large', 'cpu', 0, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:      304.974 s] (100.00%)
[  amgcl:       76.428 s] ( 25.06%)
[    setup:      9.735 s] (  3.19%)
[      full:     9.735 s] (  3.19%)
[    solve:     66.693 s] ( 21.87%)
[  reading:    228.442 s] ( 74.91%)

Average iterations: 15.4


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:      482.308 s] (100.00%)
[  amgcl:      252.266 s] ( 52.30%)
[    setup:      4.863 s] (  1.01%)
[      full:     4.863 s] (  1.01%)
[    solve:    247.403 s] ( 51.30%)
[  reading:    229.940 s] ( 47.67%)

Average iterations: 46.1
Total speedup: -69.70%
Setup speedup: 100.19%


In [11]:
report('large', 'cpu', 0, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:      304.974 s] (100.00%)
[  amgcl:       76.428 s] ( 25.06%)
[    setup:      9.735 s] (  3.19%)
[      full:     9.735 s] (  3.19%)
[    solve:     66.693 s] ( 21.87%)
[  reading:    228.442 s] ( 74.91%)

Average iterations: 15.4


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:         301.722 s] (100.00%)
[  amgcl:          75.993 s] ( 25.19%)
[    setup:         7.892 s] (  2.62%)
[      full:        0.978 s] (  0.32%)
[      partial:     6.914 s] (  2.29%)
[    solve:        68.101 s] ( 22.57%)
[  reading:       225.621 s] ( 74.78%)

Average iterations: 15.6
Total speedup: 0.57%
Setup speedup: 23.35%


## Large problem 0, VexCL(CUDA)

In [12]:
report('large', 'gpu', 0, 'full')

*** Full AMGCL rebuild on every step ***

[Profile:      260.192 s] (100.00%)
[  amgcl:       35.784 s] ( 13.75%)
[    setup:     15.417 s] (  5.93%)
[      full:    15.417 s] (  5.93%)
[    solve:     20.367 s] (  7.83%)
[  reading:    224.316 s] ( 86.21%)

Average iterations: 15.4


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:      298.334 s] (100.00%)
[ self:          7.178 s] (  2.41%)
[  amgcl:       68.389 s] ( 22.92%)
[    setup:      7.751 s] (  2.60%)
[      full:     7.751 s] (  2.60%)
[    solve:     60.638 s] ( 20.33%)
[  reading:    222.767 s] ( 74.67%)

Average iterations: 45.9
Total speedup: -47.68%
Setup speedup: 98.90%


In [13]:
report('large', 'gpu', 0, 'part')

*** Full AMGCL rebuild on every step ***

[Profile:      260.192 s] (100.00%)
[  amgcl:       35.784 s] ( 13.75%)
[    setup:     15.417 s] (  5.93%)
[      full:    15.417 s] (  5.93%)
[    solve:     20.367 s] (  7.83%)
[  reading:    224.316 s] ( 86.21%)

Average iterations: 15.4


*** Full AMGCL rebuild on every 10th step, partial rebuild elsewhere ***

[Profile:         259.004 s] (100.00%)
[  amgcl:          33.555 s] ( 12.96%)
[    setup:        13.056 s] (  5.04%)
[      full:        1.622 s] (  0.63%)
[      partial:    11.434 s] (  4.41%)
[    solve:        20.498 s] (  7.91%)
[  reading:       225.354 s] ( 87.01%)

Average iterations: 15.6
Total speedup: 6.64%
Setup speedup: 18.08%
